In [34]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder \
.master("local[*]") \
.config("spark.driver.memory", "6g") \
.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/14 07:17:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
! pwd

/


In [4]:
df = spark.read.format("parquet").load("file:///part-00096-5bd1507a-125d-4ec6-8794-3ed02e45a45d.c000.snappy.parquet")

In [5]:
df.limit(5).toPandas()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-04 23:38:18,2024-01-05 00:05:16,1.0,4.47,1.0,N,163,239,2,25.4,1.0,0.5,0.00,0.0,1.0,30.40,2.5,0.00
1,2,2024-01-04 02:21:06,2024-01-05 00:00:00,1.0,12.86,1.0,N,70,219,1,49.9,6.0,0.5,5.74,0.0,1.0,63.14,0.0,0.00
2,2,2024-01-04 08:56:48,2024-01-05 00:00:00,1.0,11.49,1.0,N,132,173,2,44.3,0.0,0.5,0.00,0.0,1.0,47.55,0.0,1.75
3,2,2024-01-04 08:16:22,2024-01-05 08:02:11,1.0,4.48,1.0,N,237,158,1,24.7,0.0,0.5,2.87,0.0,1.0,31.57,2.5,0.00
4,2,2024-01-04 08:55:16,2024-01-05 00:00:00,2.0,11.43,1.0,N,243,242,2,48.5,0.0,0.5,0.00,0.0,1.0,50.00,0.0,0.00


In [ ]:
DimTime:
TimeID
Date
DayOfMonth
WeekOfYear
Month
Year
Hour
Minute
Second

In [13]:
def write_dimtime(spark: SparkSession, table_name: str = "dimtime", start_date: str = "2022-01-01",
                  end_date: str = "2024-12-31"):
    try:
        # Convert string dates to timestamp
        start_ts = F.to_timestamp(F.lit(start_date))
        end_ts = F.to_timestamp(F.lit(end_date))

        # Generate minute-level timestamps using Spark
        dimtime_df = spark.sql(f"""
            SELECT explode(sequence(
                to_timestamp('{start_date}'), 
                to_timestamp('{end_date}'), 
                interval 1 minute
            )) as minutes
        """)

    except Exception as e:
        logger.exception(f"Error generating DataFrame: {str(e)}")

    dimtime_df2 = (dimtime_df.withColumn("unix_time_minutes", F.round(F.unix_timestamp(F.col("minutes")) / 60) * 60)
                   .withColumn("timeid", F.col("unix_time_minutes").cast("integer"))
                   .withColumn("Date", F.to_date("minutes"))
                   .withColumn("dayofmonth", F.dayofmonth("minutes"))
                   .withColumn("weekofyear", ((F.dayofyear("minutes") - 1) / 7 + 1).cast("int"))
                   .withColumn("month", F.month("minutes").cast("smallint"))
                   .withColumn("year", F.year("minutes").cast("smallint"))
                   .withColumn("minute", F.minute("minutes").cast("smallint"))
                   .withColumn("second", F.second("minutes").cast("smallint"))
                   ).drop("minutes", "unix_time_minutes")
    return dimtime_df2

In [14]:
dimtime_df = write_dimtime(spark, "dimtime", "2022-01-01", "2025-12-31")

In [18]:
dimtime_df.sample(fraction=0.1).orderBy(F.rand()).limit(20).toPandas()

25/06/14 07:35:49 WARN DAGScheduler: Broadcasting large task binary with size 16.4 MiB


,timeid,Date,dayofmonth,weekofyear,month,year,minute,second
0,1701977340,2023-12-07,7,49,12,2023,29,0
1,1740542640,2025-02-26,26,9,2,2025,4,0
2,1665980400,2022-10-17,17,42,10,2022,20,0
3,1696747680,2023-10-08,8,41,10,2023,48,0
4,1642665180,2022-01-20,20,3,1,2022,53,0
5,1658107920,2022-07-18,18,29,7,2022,32,0
6,1760480760,2025-10-14,14,41,10,2025,26,0
7,1751737080,2025-07-05,5,27,7,2025,38,0
8,1641468540,2022-01-06,6,1,1,2022,29,0
9,1702381380,2023-12-12,12,50,12,2023,43,0


In [19]:
dimtime_df.count()

25/06/14 07:36:15 WARN DAGScheduler: Broadcasting large task binary with size 16.4 MiB


2102401

In [ ]:
DimLocation:
LocationID
Borough
Zone
ServiceZone

In [20]:
! wget https://raw.githubusercontent.com/erkansirin78/datasets/00583d127f0cc780bc85c9a033d9e895b10bc4a3/nyc_taxi_yellow_trip_raw/taxi_zone_lookup.csv

--2025-06-14 07:41:11--  https://raw.githubusercontent.com/erkansirin78/datasets/00583d127f0cc780bc85c9a033d9e895b10bc4a3/nyc_taxi_yellow_trip_raw/taxi_zone_lookup.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12065 (12K) [text/plain]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  11.78K  --.-KB/s    in 0s      

2025-06-14 07:41:11 (36.5 MB/s) - ‘taxi_zone_lookup.csv’ saved [12065/12065]



In [22]:
def write_dimlocation(spark: SparkSession, path: str = None, table_name: str = "dimlocation"):
    # Read txi zones
    taxi_zones_df = spark.read.csv(path, header=True, inferSchema=True)

    return taxi_zones_df

In [23]:
taxi_zones_df = write_dimlocation(spark=spark, path="/taxi_zone_lookup.csv")

In [24]:
taxi_zones_df.limit(10).toPandas()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
5,6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
6,7,Queens,Astoria,Boro Zone
7,8,Queens,Astoria Park,Boro Zone
8,9,Queens,Auburndale,Boro Zone
9,10,Queens,Baisley Park,Boro Zone


In [30]:
df.join(taxi_zones_df, df.PULocationID == taxi_zones_df.LocationID, how='inner').limit(10).toPandas()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,LocationID,Borough,Zone,service_zone
0,2,2024-01-04 23:38:18,2024-01-05 00:05:16,1.0,4.47,1.0,N,163,239,2,...,0.00,0.0,1.0,30.40,2.5,0.00,163,Manhattan,Midtown North,Yellow Zone
1,2,2024-01-04 02:21:06,2024-01-05 00:00:00,1.0,12.86,1.0,N,70,219,1,...,5.74,0.0,1.0,63.14,0.0,0.00,70,Queens,East Elmhurst,Boro Zone
2,2,2024-01-04 08:56:48,2024-01-05 00:00:00,1.0,11.49,1.0,N,132,173,2,...,0.00,0.0,1.0,47.55,0.0,1.75,132,Queens,JFK Airport,Airports
3,2,2024-01-04 08:16:22,2024-01-05 08:02:11,1.0,4.48,1.0,N,237,158,1,...,2.87,0.0,1.0,31.57,2.5,0.00,237,Manhattan,Upper East Side South,Yellow Zone
4,2,2024-01-04 08:55:16,2024-01-05 00:00:00,2.0,11.43,1.0,N,243,242,2,...,0.00,0.0,1.0,50.00,0.0,0.00,243,Manhattan,Washington Heights North,Boro Zone
5,2,2024-01-04 09:14:56,2024-01-05 00:00:00,6.0,13.60,1.0,N,138,88,1,...,10.04,0.0,1.0,76.99,2.5,1.75,138,Queens,LaGuardia Airport,Airports
6,2,2024-01-04 10:03:30,2024-01-05 09:04:46,1.0,2.14,1.0,N,230,142,1,...,0.00,0.0,1.0,26.60,2.5,0.00,230,Manhattan,Times Sq/Theatre District,Yellow Zone
7,2,2024-01-04 11:08:33,2024-01-05 11:00:59,1.0,1.70,1.0,N,186,229,1,...,1.96,0.0,1.0,21.56,2.5,0.00,186,Manhattan,Penn Station/Madison Sq West,Yellow Zone
8,2,2024-01-04 11:54:43,2024-01-05 11:22:31,6.0,1.74,1.0,N,238,143,1,...,3.50,0.0,1.0,21.00,2.5,0.00,238,Manhattan,Upper West Side North,Yellow Zone
9,2,2024-01-04 11:58:57,2024-01-05 11:47:22,1.0,5.45,1.0,N,261,163,1,...,3.43,0.0,1.0,37.73,2.5,0.00,261,Manhattan,World Trade Center,Yellow Zone


In [ ]:
DimPayment:
PaymentTypeID
PaymentType

In [31]:
df.groupBy("payment_type").count().show()

+------------+-----+
|payment_type|count|
+------------+-----+
|           1|79376|
|           3|  758|
|           2|17911|
|           4| 1741|
+------------+-----+



In [32]:
def write_pyment_type(spark):
    data = [
        (1, "Credit card"),
        (2, "Cash"),
        (3, "No charge"),
        (4, "Dispute"),
        (5, "Unknown"),
        (6, "Voided trip")
    ]

    # Define schema
    schema = StructType([
        StructField("PaymentTypeID", IntegerType(), False),
        StructField("PaymentType", StringType(), False)
    ])
    
    # Create DataFrame
    payment_type_df = spark.createDataFrame(data, schema)

    return payment_type_df

In [35]:
payment_type_df = write_pyment_type(spark)

In [36]:
payment_type_df.show()

+-------------+-----------+
|PaymentTypeID|PaymentType|
+-------------+-----------+
|            1|Credit card|
|            2|       Cash|
|            3|  No charge|
|            4|    Dispute|
|            5|    Unknown|
|            6|Voided trip|
+-------------+-----------+



In [ ]:
DimRateCode:
RateCodeID
RateDescription

In [41]:
def write_ratecode(spark: SparkSession):
    # Data extracted from the image
    data = [
        (1, "Standard rate"),
        (2, "JFK"),
        (3, "Newark"),
        (4, "Nassau or Westchester"),
        (5, "Negotiated fare"),
        (6, "Group ride"),
        (99, "Unknown")
    ]

    # Define schema
    schema = StructType([
        StructField("RateCodeID", IntegerType(), False),
        StructField("RateDescription", StringType(), False)
    ])

    # Create DataFrame
    df = spark.createDataFrame(data, schema)
    return df

In [42]:
ratecode_df = write_ratecode(spark)

In [43]:
ratecode_df.show()

+----------+--------------------+
|RateCodeID|     RateDescription|
+----------+--------------------+
|         1|       Standard rate|
|         2|                 JFK|
|         3|              Newark|
|         4|Nassau or Westche...|
|         5|     Negotiated fare|
|         6|          Group ride|
|        99|             Unknown|
+----------+--------------------+



In [40]:
df.groupBy("RatecodeID").count().show()

+----------+-----+
|RatecodeID|count|
+----------+-----+
|       1.0|94494|
|       4.0|  188|
|       3.0|  267|
|       2.0| 3299|
|      99.0|  917|
|       5.0|  621|
+----------+-----+

